In [76]:
import pandas as pd

In [77]:
analys_teaching_record_personal_access = pd.read_csv('C:/Users/franj/Desktop/PAI UEX Repositorios/education_drop/Data/For_Analysis_and_Modeling/2nd_process/personal_access_data/pred_analys_teaching_record_personal_access.csv',
                                                       sep = '|')

In [78]:
analys_teaching_record_personal_access.head()

,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,sexo,anio_nacimiento,nota_admision_def,expediente,cod_plan,anio_nacimiento_interval,1st_model
0,GRADO EN EDUCACIÓN SOCIAL,2010-11,0,JUN,Selectividad,D,1992,8.154,110,715,"(1990.0, 1995.0]",0
1,GRADO EN EDUCACIÓN SOCIAL,2011-12,1,JUN,Selectividad,H,1992,5.682,308,715,"(1990.0, 1995.0]",1
2,GRADO EN EDUCACIÓN SOCIAL,2012-13,0,JUN,Selectividad,D,1994,11.844,322,715,"(1990.0, 1995.0]",0
3,GRADO EN EDUCACIÓN SOCIAL,2012-13,0,JUN,Selectividad,D,1994,9.126,324,715,"(1990.0, 1995.0]",0
4,GRADO EN EDUCACIÓN SOCIAL,2012-13,1,JUN,Selectividad,D,1993,8.118,328,715,"(1990.0, 1995.0]",0


In [79]:
pr_teaching_plan_subject_call = pd.read_csv('../../../../../Data/Processed/Teaching/Plan_Subject_Call/pr_teaching_plan_subject_call.csv', sep='|')

In [80]:
pr_teaching_plan_subject_call.head()

,expediente,cod_plan,des_plan,cod_asignatura,des_asignatura,curso_asignatura,semestre,tipo_linea,tipologia_asignatura,des_tipogia_asignatura,curso_academico,convocatoria,nota_alf,nota_num,des_nota,numero_convocatorias_agotadas,fin_estudios
0,4,715,GRADO EN EDUCACIÓN SOCIAL,500445,ANTROPOLOGÍA SOCIAL,1,1S,O,T,TRONCAL,2009-10,FEB,NT,8.0,NOTABLE,1,S
1,4,715,GRADO EN EDUCACIÓN SOCIAL,500446,FUNDAMENTOS TEÓRICOS DE LA EDUCACIÓN,1,1S,O,T,TRONCAL,2009-10,FEB,AP,5.5,APROBADO,1,S
2,4,715,GRADO EN EDUCACIÓN SOCIAL,500447,PERSPECTIVA HISTORICA Y POLÍTICA DE LA EDUCACI...,1,1S,O,T,TRONCAL,2009-10,FEB,AP,6.5,APROBADO,1,S
3,4,715,GRADO EN EDUCACIÓN SOCIAL,500448,PSICOLOGÍA DEL CICLO VITAL,1,1S,O,T,TRONCAL,2009-10,FEB,NT,7.5,NOTABLE,1,S
4,4,715,GRADO EN EDUCACIÓN SOCIAL,500449,SOCIOLOGÍA GENERAL,1,1S,O,T,TRONCAL,2009-10,FEB,AP,6.1,APROBADO,1,S


In [81]:
pr_teaching_scholarship_per_year = pd.read_csv('../../../../../Data/Processed/Teaching/Scolarship_per_year/pr_teaching_scholarship_per_year.csv', sep='|')

In [82]:
pr_teaching_scholarship_per_year.head()

,expediente,curso_academico,cod_plan,des_plan,becario
0,0,2016-17,725,MÁSTER UNIV. DE ENSEÑANZA PORTUGUÉS LENG. EXTR...,N
1,1,2009-10,717,GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),S
2,1,2010-11,717,GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),N
3,1,2011-12,717,GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),N
4,1,2012-13,717,GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),N


In [97]:
def get_cum_p_data(p, curso:int, semestre:str):
    p_data = final_plan_subject_call[(final_plan_subject_call['cod_plan'] == p.cod_plan) &
                   (final_plan_subject_call['expediente'] == p.expediente)
                                ].sort_values(by=['curso_academico'])
    courses = p_data['curso_academico'].unique()[0:curso]

    p_data = p_data[p_data['curso_academico'].isin(courses)]
    if len(p_data.index > 0):
        if(semestre == '1S'):
            no_valid_data = p_data[(p_data['curso_academico']==courses[curso-1]) & 
                           (p_data['semestre'] == '2S')].index
            p_data = p_data[p_data.index.isin(no_valid_data) == False]

    return(p_data)
    

In [98]:
def get_course_p_data(p, curso:int):
    p_data = final_scolarship_per_year[(
        final_scolarship_per_year['cod_plan'] == p.cod_plan) 
        &(final_scolarship_per_year['expediente'] == p.expediente)
    ].sort_values(by=['curso_academico'])
    
    course = p_data['curso_academico'].unique()[curso-1]
    p_data = p_data[p_data['curso_academico'] == course]
    return(p_data)

In [99]:
def get_cum_core_ratio(p, curso:int, semestre:str):
    p_data = get_cum_p_data(p, curso, semestre)
        
    n_core_subjects  = len(p_data[p_data['tipologia_asignatura'] == 'T'].index)
    n_passed_core_subjects = len(p_data[(p_data['tipologia_asignatura'] == 'T') &
                                        (p_data['nota_num'] >= 5.0)].index)
    if n_core_subjects > 0:
    
        return (n_passed_core_subjects / n_core_subjects)
    else:
        return -1

In [100]:
def get_cum_median(p, curso:int, semestre:str):
    import numpy as np
    p_data = get_cum_p_data(p, curso, semestre)
    return np.nanmedian(p_data['nota_num'])
    

In [101]:
def get_semester_median(p, curso:int, semestre:str):
    import numpy as np
    p_data = get_course_p_data(p, curso)
    p_data = p_data[p_data['semestre']==semestre]
    return np.nanmedian(p_data['nota_num'])


In [102]:
def get_cum_absent_ratio(p, curso:int, semestre:str):
    p_data = get_cum_p_data(p, curso, semestre)
    
    n_subjects = len(p_data.index)
    n_absent_subjects = len(p_data[p_data['des_nota']=='NO PRESENTADO'].index)
    
    return (n_absent_subjects/n_subjects)

In [103]:
def get_scolarship(p, curso:int):
    p_data = get_course_p_data(p, curso)
    return (p_data['becario'].values[0])


In [104]:
analys_teaching_record_personal_access['cum_core_ratio'] = analys_teaching_record_personal_access.apply(
lambda func: get_cum_core_ratio(func,1,'1S'), axis=1)

In [105]:
len(analys_teaching_record_personal_access.index)

1985

**Aquellos datos con el valor a -1 es porque tenemos sus datos a partir del 2º curso**

In [106]:
analys_teaching_record_personal_access = analys_teaching_record_personal_access[analys_teaching_record_personal_access['cum_core_ratio'] != -1]

In [107]:
len(analys_teaching_record_personal_access.index)

1985

In [108]:
analys_teaching_record_personal_access['scolarship'] = analys_teaching_record_personal_access.apply(
lambda func: get_scolarship(func,1), axis=1)

In [109]:
analys_teaching_record_personal_access['cum_absent'] = analys_teaching_record_personal_access.apply(
lambda func: get_cum_absent_ratio(func,1,'1S'), axis=1)

In [110]:
analys_teaching_record_personal_access['cum_median'] = analys_teaching_record_personal_access.apply(
lambda func: get_cum_median(func,1,'1S'), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarnin

In [111]:
analys_teaching_record_personal_access.isnull().sum()

des_plan                     0
anio_apertura_expediente     0
abandona                     0
convocatoria_acceso          0
des_acceso                   0
sexo                         0
anio_nacimiento              0
nota_admision_def            0
expediente                   0
cod_plan                     0
anio_nacimiento_interval     0
1st_model                    0
cum_core_ratio               0
scolarship                   0
cum_absent                   0
cum_median                  17
dtype: int64

In [112]:
analys_teaching_record_personal_access.to_csv('../../../../../Data/For_Analysis_and_Modeling/2nd_process/1st_quadrimester/analys_teaching_record_personal_access.csv', sep='|', index=False)